In [6]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls 
import os

In [7]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/"#proportion_pxt_neh/"
time=31
pw=0.046

In [49]:
data=pd.read_csv(path+"__CPSM_MOUSE.csv")
data=data[data.exp_Time==time]
data=data[data.exp_Time==time].reset_index(drop="True")
print(data.shape)
data.head()

(40, 27)


,Peptied,charge,exp_Time,NEH,e_NEH,pxt,e_pxt,mo,m1,m2,...,i4,i5,NEH_new,new_error,new_cor,new_rsq,map,mean,median,sd
0,SMEYcIIEVNAR,2,31,25,22,0.046,0.050,0.40995,0.31550,0.17244,...,0.105282,0.039646,27,0.016139,0.991109,0.963540,0.097301,0.014946,0.015117,0.006090
1,IEFEGQSVDFVDPNK,2,31,28,27,0.046,0.050,0.36570,0.34991,0.18540,...,0.134417,0.063831,38,0.030845,0.918950,0.820631,0.211080,0.028196,0.028799,0.012506
2,TAVDSGIALLTNFQVTK,2,31,25,26,0.046,0.045,0.35488,0.35097,0.19026,...,0.116566,0.054945,26,0.005634,0.999489,0.994609,0.030658,0.005004,0.004914,0.002589
3,EPLFGISTGNIITGLAAGAK,3,31,36,32,0.046,0.048,0.32091,0.34919,0.20558,...,0.162041,0.088454,38,0.018600,0.981052,0.923551,0.119589,0.016103,0.017656,0.009309
4,PSYVLSGSAMNVVFSEDEMK,2,31,36,32,0.046,0.048,0.26161,0.31262,0.22573,...,0.186275,0.120335,28,0.023917,0.944173,0.858047,0.147083,0.020110,0.018683,0.012947


In [50]:
res=[]
for pep_index in range(data.shape[0]):
    exp_0=list(data.loc[pep_index,["mo","m1","m2","m3","m4","m5"]])
    b=list(data.loc[pep_index,["io","i1","i2","i3","i4","i5"]])#exp_t
    
#     if (data.loc[pep_index,"Peptied"]!="IEFEGQSVDFVDPNK"):
#         continue
    
    A=[]
    for i in range(6):
        A.append(([0]*i)+exp_0[:6-i])
    A=np.array(A).transpose()
#     x=np.matmul( np.linalg.inv ( np.matmul(A.transpose(),A)) , np.matmul(A.transpose(),b)  )
    x,residual=nnls(A,b)
    
#     ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
#     residual=sum([ abs(b[i]-y) for index,y in enumerate(ypred)])
    
    ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
    sigma=sum([ (b[i]-y)**2 for index,y in enumerate(ypred)])/(len(ypred)-1)
    cov=((sigma)*np.linalg.inv(np.matmul(A.transpose(),A)))
    var=[cov[x][x] for x in range(6)]
    
    print("\n",data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],"\n",np.corrcoef(x,exp_0)[1][0])
    
    neh= (x[2]/x[1])*2* ((1-pw)/pw) +1
    xm=sum([(index)*val / sum(x) for index,val in enumerate(x)])
    res.append([data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],data.loc[pep_index,"NEH"],neh,xm, 
               data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"],residual])
    
    r=x[1]/x[2]
    c=data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"]  #0.0610440961749121
    t_pw= (r*c-2)/ (r-2)
    print(pw,neh,pw*neh,"\t",c,"\t",t_pw,c/t_pw)
#     print(data.loc[pep_index,"Peptied"],x,end="\n\n")


 SMEYcIIEVNAR 2 
 0.9312473350992816
0.046 26.974985831191702 1.2408493482348182 	 1.0999993607401855 	 0.6039043043075771 1.821479583592999

 IEFEGQSVDFVDPNK 2 
 0.9131510898445198
0.046 38.1634308512112 1.7555178191557153 	 1.3499992154538638 	 0.5580524468421995 2.419126057224516

 TAVDSGIALLTNFQVTK 2 
 0.9873351958368588
0.046 26.019642280964234 1.1969035449243548 	 1.1699994653463377 	 0.1763505832121446 6.634508625008983

 EPLFGISTGNIITGLAAGAK 3 
 0.9073094667014923
0.046 37.49390651273001 1.7247196995855802 	 1.535999178886413 	 0.2944262216826155 5.216923853141666

 PSYVLSGSAMNVVFSEDEMK 2 
 0.9463768981433565
0.046 27.60947024508426 1.2700356312738759 	 1.535999178886413 	 -0.8936138695675405 -1.7188622862688459

 TAVDSGIALLTNFQVTK 3 
 0.9855116606113522
0.046 27.295243493221 1.255581200688166 	 1.1499993316829211 	 0.4401021591564476 2.6130281521161978

 AFAISGPFNVQFLVK 2 
 0.9938683495516929
0.046 21.254327672062576 0.9776990729148785 	 1.0499993897974498 	 3.138898427168556

In [51]:
sum([(index)* (val / sum(x)) for index,val in enumerate(x)])


1.1511875998810779

In [52]:
res=pd.DataFrame(res)
res.columns=["Peptide","Charge","True NEH","e NEH","sum_xi","pxt_neh","residual"]
res["dif"]=abs(res["True NEH"]- res["e NEH"])
# res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]
res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]

res=res.sort_values(by="dif")
print(res.shape)
res[["Peptide","Charge","True NEH","e NEH","dif"]].head(50)

(40, 9)


,Peptide,Charge,True NEH,e NEH,dif
9,FDEHGIMVLGcGPY,2,23,23.018738,0.018738
15,QAFITAQNHGY,2,24,23.960527,0.039473
10,TSAcFEPSLDYMVTK,2,23,23.108664,0.108664
28,QNLIAEVSTK,2,19,19.130300,0.130300
20,VSQEHPVVLTK,3,19,18.695595,0.304405
21,SAYALGGLGSGIcPNK,3,30,29.191176,0.808824
2,TAVDSGIALLTNFQVTK,2,25,26.019642,1.019642
27,DGSIDLVINLPNNNTK,2,23,24.085994,1.085994
31,VAQAPWK,2,15,13.798288,1.201712
23,FLGVAEQLHNEGFK,3,27,28.213227,1.213227


In [53]:
res[["Peptide","Charge","True NEH","e NEH","dif"]].tail(30)

,Peptide,Charge,True NEH,e NEH,dif
29,cEMASTGEVAcFGEGIHTAFLK,3,43,44.287557,1.287557
3,EPLFGISTGNIITGLAAGAK,3,36,37.493907,1.493907
32,GQNQPVLNITNR,2,24,22.383799,1.616201
11,EPLFGISTGNIITGL,2,22,20.270943,1.729057
25,FLGVAEQLHNEGFK,2,27,25.223080,1.776920
19,IMGTSPLQIDR,2,20,18.136304,1.863696
0,SMEYcIIEVNAR,2,25,26.974986,1.974986
26,IAPSFAVESMEDALK,2,33,31.018782,1.981218
13,NLAVPLYK,2,11,8.804447,2.195553
33,AADTIGYPVMIR,2,22,19.794626,2.205374


In [54]:
res.to_csv("testx2x1.csv",index=False)

In [55]:
# temp=pd.read_csv("C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/pw_known_search_neh/__CPSM_MOUSE.csv")
# temp["dif"]=temp.NEH-temp.e_NEH
# temp.sort_values(by="Peptied") 

In [56]:
all_AAs=['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p','q', 'r', 's', 't', 'v', 'w', 'y']
def charcount(pepSeq):
    res={key: 0 for key in all_AAs}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [57]:
def sort_pep(peptidesinfo,peptides):
    temp={p: peptidesinfo[p]["T_NEH"] for i,p in enumerate(peptides)}
    temp=dict(sorted(temp.items(), key=lambda item: item[1])) 
    return list(temp.keys()) 

In [58]:
# peptides=[x  for x in peptides if 'A' in x]
peptidesinfo=dict()
peptides=[]
for k in range(len(res)):
    peptides.append(res.loc[k,"Peptide"]+"_"+str(res.loc[k,"Charge"]))
    peptidesinfo[peptides[-1]]={"T_NEH":float(res.loc[k,"True NEH"]),"Seq":res.loc[k,"Peptide"],
                           "charcount":charcount(res.loc[k,"Peptide"])}

visited=set()
visited.add(peptides[0].split('_')[1])
coefficients=[]
peptides=sort_pep(peptidesinfo,peptides)
for k in range(len(peptides)):

    pep1=peptidesinfo[peptides[k]]  
    visited.add(peptides[k].split('_')[1])
 
    coefficients.append([pep1["charcount"][c] for c in all_AAs]+[pep1["T_NEH"]])

In [72]:
coefficients=np.array(coefficients)
sol=list(nnls(coefficients[:,:-1],coefficients[:,-1])[0])
# np.array(coefficients).shape

In [74]:

res=pd.DataFrame()
res["AA"]=all_AAs
res["True_value"]=[4.0,1.62,1.89,3.95,0.32,2.06,2.88,1.0,0.6,0.54,1.12,1.89,2.59,3.95,3.43,2.61,0.2,0.56,0.08,0.42]
res["est_value"]=sol
res

,AA,True_value,est_value
0,a,4.00,3.921627
1,c,1.62,1.653100
2,d,1.89,1.987506
3,e,3.95,3.939526
4,f,0.32,0.379819
5,g,2.06,2.025658
6,h,2.88,2.590589
7,i,1.00,0.957011
8,k,0.60,0.268208
9,l,0.54,0.590165
